In [1]:
import pandas as pd
import pyarrow
import numpy as np
from sf_conn_2 import Config
import snowflake.connector as sfc
from sql_transformations import spotify_transformation

In [8]:
def build_refined_playlist():
    spotify_transformation_client = spotify_transformation()
    spotify_transformation_client.refined_playlists()

In [8]:
conn = Config().create_sf_conn()
cursor = conn.cursor()
try:
    
    query = """
                CREATE OR REPLACE TRANSIENT TABLE ANALYTICS_JUSTIN.SPOTIFY_SCHEMA.MAIN_SPOTIFY AS
                        SELECT 
                            p.track_id,
                            p.playlist_id,
                            p.playlist_name,
                            p.track_name,
                            i.track_popularity,
                            p.artists,
                            p.genres,
                            a.danceability,
                            a.duration_ms,
                            a.energy,
                            a.instrumental,
                            a.song_key,
                            a.liveness,
                            a.loudness,
                            a.speechiness
                        FROM ANALYTICS_JUSTIN.SPOTIFY_SCHEMA.ALL_PLAYLISTS_REFINED p
                        INNER JOIN ANALYTICS_JUSTIN.SPOTIFY_SCHEMA.AUDIO_FEATURES_REFINED a ON p.track_id = a.track_id
                        INNER JOIN ANALYTICS_JUSTIN.SPOTIFY_SCHEMA.TRACK_INFORMATION_REFINED i ON p.track_id = i.track_id
            """
    cursor.execute(query)
except sfc.Error as e: 
    print(f'Error happened in TRACK_INFORMATION_REFINED: {e}')
finally:
    cursor.close()
    conn.close()